In [ ]:
!pip install -U scikit-image


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 45.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.21.0
    Uninstalling scikit-image-0.21.0:
      Successfully uninstalled scikit-image-0.21.0


In [29]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.feature import graycomatrix, graycoprops
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
import time
from tabulate import tabulate

drive.mount('/content/drive')

base_path = '/content/drive/MyDrive/Data-tangan'

def load_images(base_path):
    class_label = []
    files_path = []
    imgs = []

    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    file_path = os.path.join(folder_path, file)
                    try:
                        img = cv2.imread(file_path)
                        if img is None:
                            continue
                        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                        gray = cv2.resize(gray, (256, 256))

                        imgs.append(gray)
                        class_label.append(folder)
                        files_path.append(file_path)
                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")

    return np.array(imgs), np.array(class_label), np.array(files_path)

print("Memuat gambar...")
imgs, class_label, files_path = load_images(base_path)
print(f"Total gambar yang dimuat: {len(imgs)}")

def calc_glcm_all_agls(img, label, c_label, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):
    glcm = graycomatrix(img,
                       distances=dists,
                       angles=agls,
                       levels=lvl,
                       symmetric=sym,
                       normed=norm)

    feature = []
    glcm_props = [property for name in props for property in graycoprops(glcm, name)[0]]
    for item in glcm_props:
        feature.append(item)

    feature.append(label)
    feature.append(c_label)
    return feature

properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']

print("Memulai ekstraksi fitur GLCM...")
start_time = time.time()

glcm_all_agls = []
for img, label, c_label in zip(imgs, files_path, class_label):
    glcm_all_agls.append(
        calc_glcm_all_agls(img,
                          c_label,
                          label,
                          props=properties)
    )

end_time = time.time()
glcm_time = end_time - start_time
print(f"Ekstraksi fitur GLCM selesai. Waktu yang dibutuhkan: {glcm_time:.2f} detik")

columns = []
angles = ['0', '45', '90', '135']
for name in properties:
    for ang in angles:
        columns.append(name + "_" + ang)
columns.append("class")
columns.append("files_name")

glcm_df = pd.DataFrame(glcm_all_agls, columns=columns)

glcm_df.to_csv("glcm_tangan.csv", index=False)
print("Fitur GLCM telah disimpan ke glcm_tangan.csv")

X = glcm_df.drop(['class', 'files_name'], axis=1)
y = glcm_df['class']

def evaluate_model(model, X_train, X_test, y_train, y_test, model_name, random_state):
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time

    start_time = time.time()
    y_pred = model.predict(X_test)
    predict_time = time.time() - start_time

    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    acc = accuracy_score(y_test, y_pred)

    return {
        'model': model_name,
        'accuracy': acc,
        'precision': report['weighted avg']['precision'],
        'recall': report['weighted avg']['recall'],
        'f1_score': report['weighted avg']['f1-score'],
        'train_time': train_time,
        'predict_time': predict_time,
        'random_state': random_state
    }

models = [
    ('SVM', SVC(kernel='rbf', C=1.0, gamma='scale')),
    ('ANN', MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)),
    ('Decision Tree', DecisionTreeClassifier())
]

test_sizes = [0.2, 0.3]
n_experiments = 5
random_states = [42, 123, 456, 789, 101112]

results = []

for test_size in test_sizes:
    split_name = f"{int((1-test_size)*100)}-{int(test_size*100)}"
    print(f"\n=== Pembagian Data {split_name} ===")

    for i in range(n_experiments):
        random_state = random_states[i]
        print(f"\nEksperimen {i+1} dengan random_state {random_state}")

        X_train, X_test, y_train, y_test = train_test_split(
            X, y,
            test_size=test_size,
            random_state=random_state,
            stratify=y
        )

        print(f"Jumlah data latih: {len(X_train)}")
        print(f"Jumlah data uji: {len(X_test)}")

        for model_name, model in models:

            if hasattr(model, 'random_state'):
                model.set_params(random_state=random_state)

            print(f"\nEvaluasi {model_name}...")
            result = evaluate_model(
                model, X_train, X_test, y_train, y_test,
                model_name, random_state
            )
            result['train_test_split'] = split_name
            result['experiment'] = i+1
            results.append(result)

            print(f"Akurasi: {result['accuracy']:.4f}")
            print(f"Waktu pelatihan: {result['train_time']:.2f} detik")

results_df = pd.DataFrame(results)

results_df.to_csv("hasil_eksperimen.csv", index=False)
print("\nHasil eksperimen telah disimpan ke hasil_eksperimen.csv")

print("\nRangkuman Hasil Eksperimen:")
summary_cols = ['model', 'train_test_split', 'experiment', 'random_state', 'accuracy', 'precision', 'recall', 'f1_score', 'train_time']
print(tabulate(results_df[summary_cols], headers='keys', tablefmt='psql'))

for split in ['80-20', '70-30']:
    print(f"\n=== Hasil untuk Pembagian Data {split} ===")
    split_df = results_df[results_df['train_test_split'] == split]

    avg_results = split_df.groupby('model').agg({
        'accuracy': ['mean', 'std'],
        'precision': ['mean', 'std'],
        'recall': ['mean', 'std'],
        'f1_score': ['mean', 'std'],
        'train_time': ['mean', 'std']
    }).reset_index()

    print(tabulate(avg_results, headers='keys', tablefmt='psql'))

with open("waktu_ekstraksi.txt", "w") as f:
    f.write(f"Waktu ekstraksi fitur GLCM: {glcm_time:.2f} detik")

drive_path = "/content/drive/MyDrive/Hasil-GLCM"
os.makedirs(drive_path, exist_ok=True)

shutil.move("glcm_tangan.csv", os.path.join(drive_path, "glcm_tangan.csv"))
shutil.move("hasil_eksperimen.csv", os.path.join(drive_path, "hasil_eksperimen.csv"))
shutil.move("waktu_ekstraksi.txt", os.path.join(drive_path, "waktu_ekstraksi.txt"))

print(f"\nSemua file hasil telah disimpan ke: {drive_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Memuat gambar...
Total gambar yang dimuat: 800
Memulai ekstraksi fitur GLCM...
Ekstraksi fitur GLCM selesai. Waktu yang dibutuhkan: 37.58 detik
Fitur GLCM telah disimpan ke glcm_tangan.csv

=== Pembagian Data 80-20 ===

Eksperimen 1 dengan random_state 42
Jumlah data latih: 640
Jumlah data uji: 160

Evaluasi SVM...
Akurasi: 0.6562
Waktu pelatihan: 0.02 detik

Evaluasi ANN...
Akurasi: 0.8313
Waktu pelatihan: 0.44 detik

Evaluasi Decision Tree...
Akurasi: 0.9938
Waktu pelatihan: 0.02 detik

Eksperimen 2 dengan random_state 123
Jumlah data latih: 640
Jumlah data uji: 160

Evaluasi SVM...
Akurasi: 0.6125
Waktu pelatihan: 0.02 detik

Evaluasi ANN...
Akurasi: 0.9125
Waktu pelatihan: 0.52 detik

Evaluasi Decision Tree...
Akurasi: 0.9812
Waktu pelatihan: 0.02 detik

Eksperimen 3 dengan random_state 456
Jumlah data latih: 640
Jumlah data uji: 160

Evaluasi SVM...
Akur

In [ ]:
!pip install scikit-image==0.21.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 41.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.18.3
    Uninstalling scikit-image-0.18.3:
      Successfully uninstalled scikit-image-0.18.3
